In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import argparse

import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [2]:
n_epochs = 1
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('batch size: {}'.format(batch_size))
print('epochs: {}'.format(n_epochs))

batch size: 64
epochs: 1


In [3]:
class z24Dataset(Dataset):
    def __init__(self, mode='training', window_size=100, normalize=True):
        self.window_size = window_size
        self.slices_per_file = 65536 // self.window_size
        self.normalize = normalize
        
        if mode == 'training':
            self.index_file = np.loadtxt('../tools/training_set_index.txt',dtype=str)
        elif mode == 'testing' :
            self.index_file = np.loadtxt('../tools/test_set_index.txt',dtype=str)
        elif mode == 'validating':
            self.index_file = np.loadtxt('../tools/validation_set_index.txt',dtype=str)
        
        self.name_index_dict = dict(zip(range(len(self.index_file)),list(self.index_file)))
        
        self.env_mean = np.load('../tools/env_mean.npy')
        self.env_std = np.load('../tools/env_std.npy')
        self.vibration_mean = np.load('../tools/vibration_mean.npy')
        self.vibration_std = np.load('../tools/vibration_std.npy')

    def __len__(self):
        return len(self.index_file) * self.slices_per_file

    def __getitem__(self, index):
        index_to_read = index // self.slices_per_file
        file_to_read = self.name_index_dict[index_to_read]
        index_in_dataframe = (index - index_to_read*self.slices_per_file) * self.window_size
        
        file_path_vib = '../data/z24_clean/'+file_to_read+'_vibrations.npy'
        file_path_env = '../data/z24_clean/'+file_to_read+'_env.npy'
        
        memmap_vib = np.memmap(file_path_vib, dtype=np.float64, mode='r', shape=(65536, 7))
        memmap_env = np.memmap(file_path_env, dtype=np.float64, mode='r', shape=(53,))

        X_environmental = np.array(memmap_env[:])
        X_vibration_window = np.array(memmap_vib[index_in_dataframe:index_in_dataframe+self.window_size,:])

        if self.normalize:
            X_vibration_window = (X_vibration_window - self.vibration_mean) / self.vibration_std
            X_environmental = (X_environmental - self.env_mean) / self.env_std
        
        X_vib_and_env = np.append(X_vibration_window.flatten(),X_environmental)
       
        return X_vib_and_env, X_vibration_window.flatten()

In [4]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, z_size, output_size, dropout_p):
        super(Model, self).__init__()
        self.dropout_p = dropout_p

        self.h1 = nn.Linear(input_size, hidden_size)
        self.z  = nn.Linear(hidden_size, z_size)
        self.h2 = nn.Linear(z_size, hidden_size)
        self.h3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.dropout(x, p=self.dropout_p, training=True)
        x = F.leaky_relu(self.h1(x))
        x = F.dropout(x, p=self.dropout_p, training=True)
        x = F.leaky_relu(self.z(x))
        x = F.dropout(x, p=self.dropout_p, training=True)
        x = F.leaky_relu(self.h2(x))
        x = F.dropout(x, p=self.dropout_p, training=True)
        x = self.h3(x)
        return x
        
class Initializer:
    # to apply xavier_uniform:
    #Initializer.initialize(model=net, initialization=init.xavier_uniform_, gain=init.calculate_gain('relu'))
    # or maybe normal distribution:
    #Initializer.initialize(model=net, initialization=init.normal_, mean=0, std=0.2)
    def __init__(self):
        pass

    @staticmethod
    def initialize(model, initialization, **kwargs):
        def weights_init(m):
            if isinstance(m, nn.Linear):
                initialization(m.weight.data, **kwargs)
                try:
                    initialization(m.bias.data)
                except:
                    pass
        model.apply(weights_init)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [5]:
#hyperparameters
dropout_p = 0.1
learning_rate = 0.001
weight_decay = 1e-6
weight_init = init.xavier_normal_
hidden_size = 500
z_size = 100

In [6]:
#define model
model = Model(input_size=7*100+53,
              hidden_size=hidden_size,
              z_size=z_size,
              output_size=7*100,
              dropout_p=dropout_p).to(device)

Initializer.initialize(model=model, initialization=weight_init)

optimizer = torch.optim.Adam(model.parameters(),
                            lr=learning_rate,
                            betas=(0.9, 0.99),
                            eps=1e-08,
                            weight_decay=0,
                            amsgrad=True)
                            

In [7]:
count_parameters(model)

828300

In [8]:
#Datasets
training_dataset = z24Dataset(mode='training', window_size=100, normalize=True)
training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

validating_dataset = z24Dataset(mode='validating', window_size=100, normalize=True)
validating_dataloader = DataLoader(validating_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [128]:
loss_full = torch.nn.MSELoss(reduce=False)

In [9]:
model = torch.load(f='../results/trained_autoencoder_memmap.pt',map_location='cpu')

/home/dsteinar/miniconda3/lib/python3.6/site-packages/torch/serialization.py:333: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


In [206]:
batchloss_train = []
for X_train, Y_train in validating_dataloader:
    X_train_tensor = X_train.float().to(device)
    Y_train_tensor = Y_train.float().to(device)
    
    batch_size, output_size = Y_train.shape
    N = 100
    N_predictions = torch.zeros([N, batch_size, output_size])
    
    for i in range(N):
        N_predictions[i,:,:] = model(X_train_tensor)
    
    prediction_mean = torch.mean(N_predictions, dim=0)
    prediction_std = torch.std(N_predictions, dim=0)

    trainloss_full = loss_full(predicted_Y, Y_train_tensor)
    
    lower_y = predicted_Y - 2*prediction_std
    upper_y = predicted_Y + 2*prediction_std
    within_lower = Y_train_tensor > lower_y
    within_upper = Y_train_tensor < upper_y
    within_range = within_lower & within_upper
    
    trainloss_full[within_range] = 0
    
    uncertainty_loss = torch.sum(trainloss_full)/ torch.numel(trainloss_full)

    batchloss_train.append(uncertainty_loss.item())

In [179]:
trainloss.item()

1.616243839263916

In [180]:
test = torch.sum(trainloss_full)/ torch.numel(trainloss_full)
test

tensor(1.6163)

In [193]:
batch_size = X_train.shape[0]

lower_y = predicted_Y - 1
upper_y = predicted_Y + 1
within_lower = Y_train_tensor > lower_y
within_upper = Y_train_tensor < upper_y
within_range = within_lower & within_upper

In [194]:
within_lower

tensor([[ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1],
        ...,
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1]], dtype=torch.uint8)

In [195]:
within_upper

tensor([[ 0,  1,  1,  ...,  1,  1,  1],
        [ 0,  1,  1,  ...,  1,  1,  1],
        [ 0,  1,  1,  ...,  1,  1,  1],
        ...,
        [ 0,  1,  1,  ...,  1,  1,  1],
        [ 0,  1,  1,  ...,  1,  1,  1],
        [ 0,  1,  1,  ...,  1,  1,  1]], dtype=torch.uint8)

In [196]:
within_range

tensor([[ 0,  1,  1,  ...,  1,  1,  1],
        [ 0,  1,  1,  ...,  1,  1,  1],
        [ 0,  1,  1,  ...,  1,  1,  1],
        ...,
        [ 0,  1,  1,  ...,  1,  1,  1],
        [ 0,  1,  1,  ...,  1,  1,  1],
        [ 0,  1,  1,  ...,  1,  1,  1]], dtype=torch.uint8)

In [197]:
within_range.sum()

tensor(20778)

In [198]:
lol = trainloss_full.clone()

In [199]:
lol[within_range] = 0

In [203]:
torch.sum(lol)

tensor(38925.3906)

In [204]:
torch.sum(trainloss_full)

tensor(41861.0156)

In [200]:
uncertainty_loss = torch.sum(lol)/ torch.numel(lol)
uncertainty_loss.detach().numpy()

array(1.5029109, dtype=float32)

In [175]:
trainloss_full.numel()

25900

In [190]:
Y_train_tensor.detach().numpy()

array([[ 2.1033847e+00, -7.7909879e-02,  9.0226103e-03, ...,
         4.1447887e-03,  7.7459201e-02, -3.4782525e-02],
       [ 2.6750982e+00,  1.6679198e-01, -7.4148878e-02, ...,
        -2.8725275e-01, -3.7195751e-01,  3.8336131e-01],
       [ 2.1140544e+00, -2.1987635e-01, -2.2084813e-03, ...,
         2.2950126e-02, -1.7995991e-01, -1.3363613e-01],
       ...,
       [ 2.2500920e+00, -1.9684860e-01, -3.9239279e-03, ...,
         3.9925534e-02,  5.2312654e-02, -1.5650688e-01],
       [ 2.0687084e+00, -7.5941764e-02,  9.3081184e-03, ...,
         5.2884631e-03,  8.5518353e-02, -4.7565304e-02],
       [ 2.4679298e+00,  4.3799764e-01, -2.3930125e-01, ...,
        -3.3142719e-01, -3.4920231e-01,  5.4056108e-01]], dtype=float32)

In [205]:
X_train.shape[0]

37

In [208]:
N_predictions.shape

torch.Size([100, 37, 700])

In [213]:
mean = torch.mean(N_predictions, dim=0)
std = torch.std(N_predictions, dim=0)

In [217]:
uncertainty_loss = torch.sum(trainloss_full)/ torch.numel(trainloss_full)

In [218]:
uncertainty_loss.item()

1.298634648323059

In [219]:
uncertainty_loss

tensor(1.2986)